![alt text](./Graphs/MySQL_logo.png "MySQL Logo")

**Disclaimer:** Last time, we learned how to log on to MySQL database as root, how to make a new database and a new user and to give that new user permission to the database. Today, we'll learn how to create and populate tables and later how to query data in tables. 

In most of the following examples, I will log on to the MySQL database 'Example' on my machine as user 'Martina' with password 'password'. If the user you have created on your MySQL database has a different name and/or password, you of course have to change these values before the code in this Jupyter Notebook will run on your machine. 

In [1]:
import mysql.connector

# configuration of user details 
# change them to your user name and password to make examples run on your machine
config = {'user': 'Martina', 'password': 'password', 'host': 'localhost', 'database': 'Example'}
# config = {'user': 'Martina', 'password': 'password', 'host': '127.0.0.1', 'database': 'Examples'} # for windows

## Creating Tables

Next, we'll learn how to create a new table in SQL. I will show you the syntax that you would use when working directly from the command line, but we'll actually create our table from within Python. 

For example, let's make a table that stores information about a person. Start by thinking about what type(s) of information you want to store and what data type each piece of information should have:

| Column | Type | Allowable values |
|:--|:--|:--|
| Name  | Varchar(40)  |   |
| Gender  | Char(1)  | M,F  |
|  DOB | Date  |   |
| Address  | Varchar(100)  |   |
| Favorite_foods  | Varchar(200)  |   |

Consider that names are typically stored in two parts (first name, last name) and that addresses commonly have a street name, street number, city name, zip code etc. Neither names nor addresses or birth-dates uniquely identify people. Therefore, none of these columns make good primary keys. We could create another column (Person_ID) that assigns each person a unique integer value. We don't know how many favorite foods each person would list. It might make sense to list possible foods in a separate table that includes a foreign key linking it to the person table. 

Here is a second attempt at a refined person table:

| Column  | Type  | Allowable values  |
|:--|:--|:--|
| Person_id  | Smallint (unsigned)  | 0 to 65,535 |
| First_name  |  Varchar(20)  |   |
| Last_name | Varchar(20)  |   |
| Gender  | Char(1)  | M, F  |
| Birth_date  | Date  |   |
|  Street | Varchar(30)  |   
| City  | Varchar(20)  |   |
| State  | Varchar(20)  |   |
| Country  | Varchar(20)  |   |
| Postal_code  | Varchar(20)  |   |

The "favorite_food" table could have the following structure:

| Column | Type  |
|---|---|
| Person_id   | Smallint (unsigned)  |
|  food |  Varchar(20) |

In this case, the two columns "Person_id" and "food" together are the primary key of the "favorite_food" table since neither uniquely identifies a row by iteself (one person could have more than one favorite food and several people might list "pizza"). Person_id is also a foreign key that links to the "person" table.  

**Note:** Right now, we have not specified how many favorite foods people can list or how they should list them. Consider what you will do with the data once it has been collected. If you want to analyze how many people prefer a certain kind of food, you may want to restrict how information is entered. Instead of allowing free responses (what do you do if one person enters "spaghetti", another enters "spagetti", and somebody else enters "pasta"?) you could allow choices from a limited set of options. 

**Creating a Table in SQL:** The syntax for creating the "person" table we defined above in SQL is 

```  
CREATE TABLE person
    (person_id SMALLINT(5) UNSIGNED,
     fname VARCHAR(20), 
     lname VARCHAR(20), 
     gender ENUM('M','F'),
     birth_date DATE, 
     street VARCHAR(30),
     city VARCHAR(20), state VARCHAR(20),
     country VARCHAR(20), postal_code VARCHAR(20),
     CONSTRAINT pk_person PRIMARY KEY (person_id));
```

The last statement here tells the database server which column will serve as the primary key for the table. That is done by creating a ```CONSTRAINT``` for the table. Here, the constraint is created on the person_id column and given the name pk_person. The ```ENUM('M','F')``` part after the definition of the gender variable tells SQL that we will only consider the possible values 'M' and 'F' for gender. Other data when entered in this field will produce an error. 

To look at the table you just created type 

```
DESC person;
```

```DESC``` is short for "describe". To delete the entire table you would type 

```
DROP TABLE person;
```

**Example:** The same table can be created from within Python. First, we will define what we want the table contents to be (as two string objects). Then, we will connect to the database and pass the table contents to MySQL. 

In [2]:
person_table = ("""                    
    CREATE TABLE person             
      (person_id SMALLINT(5) UNSIGNED,
      fname VARCHAR(20),
      lname VARCHAR(20),
      gender ENUM('M','F'),
      birth_date DATE,
      street VARCHAR(30),
      city VARCHAR(20),
      state VARCHAR(20),
      country VARCHAR(20),
      postal_code VARCHAR(20),
      CONSTRAINT pk_person PRIMARY KEY (person_id))""")
            # this is one long string to be passed to SQL
            # the use of the triple-quotes allows line breaks to be included in the string. 

favorite_food_table = ("""CREATE TABLE favorite_food
    (person_id SMALLINT(5) UNSIGNED, 
    `food` VARCHAR(20),
    CONSTRAINT pk_favorite_food PRIMARY KEY (`person_id`, `food`),
    CONSTRAINT fk_fav_food_person_id 
        FOREIGN KEY (`person_id`) REFERENCES person (`person_id`))""")
            # another string defines the favorite foods table

**Note:** In the ```favorite_food_table``` we are making both the person_id and the food together into primary keys that uniquely identify each row. That assumes that each person lists each kind of favorite food only once. In addition there is a second ```CONSTRAINT``` in the ```favorite_food_table```. This is a foreign key constraint. This means that the values in person_id in the ```favorite_food_table``` are only allowed to be a subset of the values that are present in the ```person_table```. That means that you cannot add data to the ```favorite_food_table``` for a person that does not already exist in the ```person_table```. 

It also means that these two tables are now linked. You won't be able to modify the ```favorite_food``` table without also potentially modifying the person table and vice versa. You can't drop a person from ```person```, for example, who has entries in ```favorite_food```. 

Next, we are passing the two strings we previously created with the statements for creating the two tables: ```person``` and ```favorite_food``` to SQL. Since I have previously already created these same tables (for the other section and to test my code), I will begin by removing the already existing tables. If you are making a brand new table, of course you could omit this step. If you are trying to create a table that already exists you will get an error from Python. 

In [3]:
cnx = mysql.connector.connect(**config)     # connect with config-user details
cursor = cnx.cursor()                       # create cursor object 

cursor.execute("DROP TABLE IF EXISTS favorite_food") # delete the food table if it already exists
cursor.execute("DROP TABLE IF EXISTS person")        # delete the person table if it already exists

cursor.execute(person_table)                # executing the "CREATE TABLE" code in SQL
cursor.execute(favorite_food_table)         # creating two new tables: person and favorite_food

cursor.close()                              # like connections, cursor objects should be closed after use
cnx.close()                                 # close connection

**Recap:** In the above code we are doing the following:

* We are connecting to an existing database as an existing user (remember to use your own user details if this code produces an error for you)
* Defining the content of two tables (person, favorite_food) as string commands in Python
* We are creating the two tables in the database by passing the string commands to SQL

**Examples:** Once you have created tables, you might want to look at the tables you have. When you use the command line, 

```
SHOW TABLES;
```

lists all tables in the database and 

```
DESC person;
```

describes the structure (names and types of variables) of the table "person". Shown below is what the command line output would look like. 

<img src="./Graphs/Person_table.png" width=600 />

 The same can be done from within Python with a little more effort.

In [4]:
cnx = mysql.connector.connect(**config) # connect with config-user details
cursor = cnx.cursor()                   # cursor object 

cursor.execute("SHOW TABLES")           # SQL command to list existing tables in a database

tables = cursor.fetchall()              # fetching the result from the command line

for table in tables:                    # showing all the tables one by one
    print(table)

cursor.close()                          # close cursor
cnx.close()                             # close connection 

('account',)
('branch',)
('business',)
('customer',)
('department',)
('employee',)
('favorite_food',)
('individual',)
('Movie',)
('officer',)
('person',)
('product',)
('product_type',)
('Rating',)
('Reviewer',)
('transaction',)


Here, the ```fetchall()``` method fetches all the rows of our query result (as a list of tuples) so that we can print them one-by-one. 

**Example:** You can also look at the structure of a specific table from within Python. 

In [5]:
cnx = mysql.connector.connect(**config) # connect with config-user details
cursor = cnx.cursor()                   

cursor.execute("DESC person")           # SQL command to show an existing table

results = cursor.fetchall()             # fetching the description of the person table

for row in results:                     # contents of the person table
    print(row)

cursor.close() 
cnx.close()

('person_id', 'smallint(5) unsigned', 'NO', 'PRI', None, '')
('fname', 'varchar(20)', 'YES', '', None, '')
('lname', 'varchar(20)', 'YES', '', None, '')
('gender', "enum('M','F')", 'YES', '', None, '')
('birth_date', 'date', 'YES', '', None, '')
('street', 'varchar(30)', 'YES', '', None, '')
('city', 'varchar(20)', 'YES', '', None, '')
('state', 'varchar(20)', 'YES', '', None, '')
('country', 'varchar(20)', 'YES', '', None, '')
('postal_code', 'varchar(20)', 'YES', '', None, '')


**Example:** Make the returned information more easily readable by converting the list of tuples obtained from the console into a Pandas DataFrame.

In [6]:
import pandas as pd

pd.DataFrame(results, columns = ['Field', 'Type', 'Null', 'Key', 'Default', 'Extra'])

,Field,Type,Null,Key,Default,Extra
0,person_id,smallint(5) unsigned,NO,PRI,None,
1,fname,varchar(20),YES,,None,
2,lname,varchar(20),YES,,None,
3,gender,"enum('M','F')",YES,,None,
4,birth_date,date,YES,,None,
5,street,varchar(30),YES,,None,
6,city,varchar(20),YES,,None,
7,state,varchar(20),YES,,None,
8,country,varchar(20),YES,,None,
9,postal_code,varchar(20),YES,,None,


**Example:** Let's interpret the results. 

The table shows us the columns that the person table contains (under ```Field```). It also shows us the data type for each column. The ```Null``` columns tells us whether an entry is required or could be omitted when data is entered into the table. In our table, the only required entry is the person_id. The ```Key``` column lists all columns (variables) that are involved in constructing the primary key. It is possible to populate entries with default values if no data is entered, but we have not specified any such default values. That's reflected in the ```Default``` column. The ```Extra``` column can store other pertinent information about variables. 

**Example:** Look at the ```favorite_food``` table we have created. 

In [7]:
cnx = mysql.connector.connect(**config) # open connection
cursor = cnx.cursor() 

cursor.execute("DESC favorite_food")   # SQL command to show an existing table

results = cursor.fetchall()            # fetching the description of the favorite_food table

cursor.close() 
cnx.close()                            # close connection

pd.DataFrame(results, columns = ['Field', 'Type', 'Null', 'Key', 'Default', 'Extra'])

,Field,Type,Null,Key,Default,Extra
0,person_id,smallint(5) unsigned,NO,PRI,None,
1,food,varchar(20),NO,PRI,None,


**Note:** that in this table both ```person_id``` and ```food``` are used in constructing the primary key. 

## Populating and Modifying Tables

**Example:** Now that we have constructed our two tables, the next step is to enter some data. In general, there are four things you might want to do with existing tables

* INSERT (enter new data into your table)
* UPDATE (modify an existing entry in your table)
* DELETE (remove an existing entry from your table)
* SELECT (retrieve (subset of) table - much more detail on that later)

### Inserting Data

To insert new data into a table, you have to specify three things:

* the name of the table 
* the names of the column(s)
* the value(s) you want to insert

Depending on how the table is defined (in particular, how the ```Null``` attribute is set) you are not required to provide values for all columns in the table.

### Generating automatic primary key counters

Recall, that our ```person``` table has primary key ```person_id```. How should we generate those primary key  values? It would make sense to use sequential values (i.e., start with 1 and then keep going with 2,3,...). You could assign a value yourself (but keep in mind that more than one user may be able to enter data into the same database and this way users would have to be careful to not accidentally assign the same ID to two different people). An alternative is to let SQL generate the primary key values for you. You do that by specifying ```AUTO_INCREMENT``` in the row that assigns the variable ```person_id```. It can also be done after the table is also created with a ```ALTER TABLE``` command. 

However, since the ```person_id``` is used as a foreign key in the ```favorite_foods``` table, we cannot modify it in the ```person``` table alone. Instead, first we have to modify the favorite foods table to drop the foreign key constraint, then we can modify ```person_id``` to make it auto-incremented and lastly we recreate the foreign key constraint. 

```
ALTER TABLE favorite_food 
    DROP FOREIGN KEY fk_fav_food_person_id,
    MODIFY person_id SMALLINT UNSIGNED;
            
ALTER TABLE person MODIFY person_id SMALLINT(5) UNSIGNED AUTO_INCREMENT;

ALTER TABLE favorite_food
    ADD CONSTRAINT fk_fav_food_person_id FOREIGN KEY (person_id)
          REFERENCES person (person_id);
```

**Note:** In a *real* database, you would want to restrict other users from accessing the data while you make a change like this. 

Or we could do the same thing in Python. The command line syntax has to be "fed" to Python as a string (wrapped in "quotes" if you need one line or wrapped in """triple quotes""" if you need more than one line). You can omit the semi-colon (;) symbol that is required in the command line when you use the ```cursor.execute()``` method. 

In [8]:
cnx = mysql.connector.connect(**config) # open connection
cursor = cnx.cursor() 

cursor.execute("""ALTER TABLE favorite_food      
    DROP FOREIGN KEY fk_fav_food_person_id,
    MODIFY person_id SMALLINT UNSIGNED""")
    # drop foreign key restriction in favorite_food
    
cursor.execute("ALTER TABLE person MODIFY person_id SMALLINT(5) UNSIGNED AUTO_INCREMENT")
    # alter primary key to auto-increment in person table
    
cursor.execute("""ALTER TABLE favorite_food
    ADD CONSTRAINT fk_fav_food_person_id FOREIGN KEY (person_id)
          REFERENCES person (person_id)""")
    # reinstate foreign key restriction in favorite_food
    
cursor.close() 
cnx.close()    # close connection 

Now look at your modified ```person``` table again:

In [9]:
cnx = mysql.connector.connect(**config)  # open connection
cursor = cnx.cursor() 

cursor.execute("DESC person")            # SQL command to show structure of an existing table

results = cursor.fetchall()              # fetch the description of the person table

cursor.close() 
cnx.close()                              # close connection

pd.DataFrame(results, columns = ['Field', 'Type', 'Null', 'Key', 'Default', 'Extra'])

,Field,Type,Null,Key,Default,Extra
0,person_id,smallint(5) unsigned,NO,PRI,None,auto_increment
1,fname,varchar(20),YES,,None,
2,lname,varchar(20),YES,,None,
3,gender,"enum('M','F')",YES,,None,
4,birth_date,date,YES,,None,
5,street,varchar(30),YES,,None,
6,city,varchar(20),YES,,None,
7,state,varchar(20),YES,,None,
8,country,varchar(20),YES,,None,
9,postal_code,varchar(20),YES,,None,


and note the additional statement ```auto_increment``` in the **Extra** column. When you now provide data for the ```person``` table, you can provide a null value for ```person_id``` and SQL will automatilly populate the field with the next largest integer.

**Example:** Inserting data

Let's add some actual data to our table. On the command line we could write

```
INSERT INTO person
-> (person_id, fname, lname, gender, birth_date)
-> VALUES (null, 'William','Turner', 'M', '1972-05-27');
```

Note, that the ```birth_date``` has to be specified in the date format "year-month-day". The "null" for ```person_id``` is SQL's way of specifying missing data. In this case, since the ```person_id``` is auto incremented and William Turner is our first entry in the table, it means that he will be assigned ```person_id = 1```.

You can look at all the data in the person table with the statement

```
SELECT * FROM person;
```

In Python you would add data to a table and then look at the table as follows:

In [10]:
cnx = mysql.connector.connect(**config)  # open connection
cursor = cnx.cursor()                   

cursor.execute("DELETE FROM person")     # deleted all current entries in the person table
cursor.execute("""INSERT INTO person (person_id, fname, lname, gender, birth_date) 
                  VALUES (null, 'William', 'Turner', 'M', '1972-05-27')""")
cnx.commit()                             # commit new data to the database
           
cursor.execute("SELECT * FROM person")   # SQL command to show all data in an existing table

row = cursor.fetchone()

while row is not None:
    print(row)
    row = cursor.fetchone()
    
cursor.close()                          
cnx.close()                              # close connection

(1, 'William', 'Turner', 'M', datetime.date(1972, 5, 27), None, None, None, None, None)


Note the ```cnx.commit()``` command after the data insertion statement. It commits the current transaction. This is only necessary if you insert or update data in any of your tables. 

**Notes:** Note, that the birth-date was passed as a string (in quotes). The date has to be in the date format specified when the table was created ('yyyy-mm-dd').

**Note:** In the previous example, we have used the ```cursor.fetchone()``` command to retrieve one line of information (at a time) from the console output. Previously, you have seen a related command ```cursor.fetchall()```. There is a third option, too.

* ```cursor.fetchone()``` will fetch one line at a time from output mysql produces (what you would see on the console) and return it as a tuple of records.
* ```cursor.fetchmany(n)``` will return n lines of output (formatted as a list of tuples)
* ```cursor.fetchall()``` will return all lines of output as a list of tuples. If there is no output, the list will be empty. 

**Example:** Suppose William has specified three favorite foods (Pizza, Cookies, and Nachos). We can enter this information into the ```favorite_foods``` table. However, recall that we placed a foreign key constraint in the favorite\_food table. If we tried to enter a person_id that does not exist in the person table, this code would not run. 

In [11]:
cnx = mysql.connector.connect(**config)         # open connection
cursor = cnx.cursor()                  

cursor.execute("DELETE FROM favorite_food")     # deleted all current entries in the favorite_food table
cursor.execute("INSERT INTO favorite_food (person_id, food)"
               "VALUES (1, 'pizza')")
cursor.execute("INSERT INTO favorite_food (person_id, food)"
               "VALUES (1, 'cookies')")
cursor.execute("INSERT INTO favorite_food (person_id, food)"
               "VALUES (1, 'nachos')")
cnx.commit()                                    # commit new data to the favorite_food table

cursor.execute("SELECT * FROM favorite_food")   # SQL command to show entries in an existing table

row = cursor.fetchone()

while row is not None:
    print(row)
    row = cursor.fetchone()

cursor.close()                                  
cnx.close()                                     # close connection

(1, 'cookies')
(1, 'nachos')
(1, 'pizza')


## Updating Data

You can modify data in an existing table. Recall, that when we first entered William Turner's information into the ```person``` table, we omitted his address. We can add it now using an UPDATE statement. The corresponding command line SQL code would be 

```
UPDATE person
-> SET street = '1225 Tremont St.',
-> city = 'Boston',
-> state = 'MA',
-> country = 'USA',
-> postal_code = '02138'
-> WHERE person_id = 1;
```

In Python, we could write

In [12]:
cnx = mysql.connector.connect(**config) # connect with config-user details
cursor = cnx.cursor()                   # cursor object 

cursor.execute("""UPDATE person           
                SET street = '1225 Tremont St.',  state = 'MA', city = 'Boston', country = 'USA', postal_code = '02138'
                WHERE person_id=1""")
cnx.commit()

cursor.execute("SELECT * FROM person")   # SQL command to show entries in an existing table

row = cursor.fetchone()

while row is not None:
    print(row)
    row = cursor.fetchone()
               
cursor.close()                          # close.cursor()
cnx.close()   

(1, 'William', 'Turner', 'M', datetime.date(1972, 5, 27), '1225 Tremont St.', 'Boston', 'MA', 'USA', '02138')


Now, William Turner has an address in our table.

## Deleting table entries

Suppose William has changed his mind and does no longer like Nachos. We could update the ```favorite_food``` table with the line 

```
DELETE FROM favorite_food 
WHERE person_id=1 AND food = 'nachos';
```

In [13]:
cnx = mysql.connector.connect(**config) # connect with config-user details
cursor = cnx.cursor()                   # cursor object 

cursor.execute("""DELETE FROM favorite_food 
                  WHERE person_id=1 AND food = 'nachos' """)

cursor.execute("SELECT * FROM favorite_food")   # SQL command to show all entries in an existing table

row = cursor.fetchone()

while row is not None:
    print(row)
    row = cursor.fetchone()
               
cursor.close()                          # close.cursor()
cnx.close()   

(1, 'cookies')
(1, 'pizza')


**Note:** The WHERE statement is quite important! Note, that here we want to delete the "Nachos" entry only for William Turner (he's the only guy in our table right now). If we omit the ```person_id=1``` statement from WHERE, we would delete Nachos for everybody. If you forget the entire WHERE statement, you delete all data in the table - oops.

## Query Statements in SQL

So far, we have learned how to do basic database administration. 

* Create and delete a database (as root user)
* Create new users and give them admission to specific databases
* Create and delete tables
* Insert, update and delete values in tables

In many applications, most of your job will likely center on retrieving data from existing tables, based on given criteria. You may have to combine information from more than one table to get what you need. This query process is what we'll study next. 

**Remark:** The tables that we've created ourselves for practice purposes are too small to write interesting queries. Therefore, I will now make a new database called "bank" and populate it with the data from the book. That's the database we'll use for the subsequent examples. If you have already imported the book data into your "Examples" database, you can also continue to work with that database. 

It's faster to do this on the command line. 

1. log in as root
2. make a new database ```CREATE DATABASE bank;```
3. Give full priviledges to existing user ```GRANT ALL PRIVILEGES ON bank.* to 'Martina'@'localhost';```
4. log out as root
5. log in as user 'Martina'
6. switch database in use ``` USE bank;```
7. import data ```source /Users/mb/SJSU/Math167PS/SQL-Example/LearningSQLExample.sql;```

**Example:** Let's take a look at what tables we now have in our bank database. 
You may have to change the user name and password on your machine!

In [14]:
config = {'user': 'Martina', 'password': 'password', 'host': 'localhost', 'database': 'bank'}
            # connect to 'bank' database as user Martina
cnx = mysql.connector.connect(**config) # connect with config-user details
cursor = cnx.cursor()                   # cursor object 

cursor.execute("""SHOW tables""")       # generate list of existing tables in daatabase bank

row = cursor.fetchone()

while row is not None:
    print(row)
    row = cursor.fetchone()
    
cursor.close()                          # close.cursor()
cnx.close()                             # close connection

('account',)
('branch',)
('business',)
('customer',)
('department',)
('employee',)
('individual',)
('officer',)
('product',)
('product_type',)
('transaction',)


**Example:** The following graph shows the tables contained in the "bank" database and the relationships between those tables. This kind of diagram is called an entity-relationship (ER) diagram. The primary key variables are designated by a key symbol. Foreign keys are marked with (FR). Dotted lines describe how foreign keys relate to another table's primary key. 

The markings at the end of the lines tell us how many objects are mapped to how many. For example, one bank branch maps to many employees. 

![alt text](./Graphs/Bank_Schema.png "Bank Schema")

## General Query Syntax

Query statements have a specific structure. They are usually constructed by combining two or more of the following clauses: 

| Clause Name  | Purpose  |
|:--|:--|
| SELECT  | Determines which columns to include in the query result |
| FROM  | Identifies the table from which to draw data and how tables should be joined*  |
| WHERE  | Filters data  |
| GROUP BY  | Groups rows by common column values  |
| HAVING  | Filters groups  |
| ORDER BY  | Sort the rows of the result by one or more columns  |

(*) We have not yet discussed how to join tables, we'll get that later! 

### The SELECT statement

**Examples:** SELECT is used to select all (\*) or some of the columns from a specific table. 
Here, we are selecting all columns of the table "department", for example. In this code we automatically name the columns of the Pandas DataFrame based on the table information provided by SQL. 

In [15]:
cnx = mysql.connector.connect(**config)             # connect 
cursor = cnx.cursor()                   

cursor.execute("""DESC department""")               # get details of table "department"
row = cursor.fetchone()

colnames = []
while row is not None:                              # make vector of variable names (column names)
    colnames.append(row[0])
    row = cursor.fetchone()
    
cursor.execute("""SELECT * FROM department""")       # get all data from department table
results = cursor.fetchall()

cursor.close()                                      
cnx.close()                                          # disconnect

pd.DataFrame(results, columns = colnames)            # make data frame and label columns appropriately

,dept_id,name
0,1,Operations
1,2,Loans
2,3,Administration
3,4,Operations
4,5,Loans
5,6,Administration


**Example:** Selecting only the columns "product_cd" and "name" from table "product". Here, we name the columns in our Pandas dataframe ourselves. 

In [16]:
cnx = mysql.connector.connect(**config)               # connect 
cursor = cnx.cursor()                   

colnames = ['product_cd', 'name']
    
cursor.execute("""SELECT product_cd, name FROM product""") # retrieve two columns of product table
results = cursor.fetchall()
    
cursor.close()                          
cnx.close()                                           # disconnect

pd.DataFrame(results, columns = colnames)             # make pretty DataFrame

,product_cd,name
0,AUT,auto loan
1,BUS,business line of credit
2,CD,certificate of deposit
3,CHK,checking account
4,MM,money market account
5,MRT,home mortgage
6,SAV,savings account
7,SBL,small business loan


**Example:** In some cases, you are only interested in retrieving a set of values (as opposed to a list which could contain duplicates). For example, if you would want to know the ID's of all customers who have accounts listed in the "account" table, you could type:

In [17]:
cnx = mysql.connector.connect(**config) # connect with config-user details
cursor = cnx.cursor()                   # cursor object 
    
cursor.execute("""SELECT cust_id FROM account""")       # generate list of existing tables in daatabase bank
results = cursor.fetchall()
    
cursor.close()                          # close.cursor()
cnx.close()  

pd.DataFrame(results, columns = ['cust_id'])

,cust_id
0,1
1,1
2,1
3,1
4,1
...,...
67,23
68,23
69,24
70,25


But then the same customer could show up more than once in your results. 

**Example:** If you only want the unique values, you could apply the set function in Python or ask for distinct results directly in SQL. Note the addition of "DISTINCT" right after the "SELECT" statement.

**Caution:** Using the "DISTINCT" keyword causes the data to be sorted by SQL. If your data set is very large, that can be slow. 

In [18]:
cnx = mysql.connector.connect(**config)                          # connect 
cursor = cnx.cursor()                   
    
cursor.execute("""SELECT DISTINCT cust_id FROM account""")       # generate list of distinct customer IDs
results = cursor.fetchall()
    
cursor.close()                          
cnx.close()                                                      # disconnect

pd.DataFrame(results, columns = ['cust_id'])                     # make DataFrame

,cust_id
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


### The FROM clause

In all examples you have seen above, the FROM clause refers to a specific permanently stored table from which to select data. However, the FROM clause can also refer to other query results (that is, a table which is the result of another query) or to a temporary table (or virtual table or view). A virtual table can be the result of a merger of two tables that are joined together by a common attribute (column variable). 

**Example:** We will discuss join statements in more detail later, but here is an example of a  query with a non-trivial FROM clause statement. We are joining the tables "employee" and "department" by the common column "dept_id" and then selecting employee id, first and last name of the employee and their department name to display as the result. 

In [19]:
cnx = mysql.connector.connect(**config)                         # connect 
cursor = cnx.cursor()                   
    
cursor.execute("""SELECT employee.emp_id, employee.fname,
                employee.lname, department.name AS dept_name
                FROM employee INNER JOIN department
                ON employee.dept_id = department.dept_id""")    # select columns from two joined tables

results = cursor.fetchall()
    
cursor.close()                         
cnx.close()                                                      # disconnect

pd.DataFrame(results, columns = ['emp_id', 'fname', 'lname', 'dept_name'])

,emp_id,fname,lname,dept_name
0,1,Michael,Smith,Administration
1,2,Susan,Barker,Administration
2,3,Robert,Tyler,Administration
3,4,Susan,Hawthorne,Operations
4,5,John,Gooding,Loans
5,6,Helen,Fleming,Operations
6,7,Chris,Tucker,Operations
7,8,Sarah,Parker,Operations
8,9,Jane,Grossman,Operations
9,10,Paula,Roberts,Operations


**Example:** Referencing and Aliasing

Note, that in the last example, we have to specify the columns we want to select as ```employee.emp_id``` for example. Here, "employee" is the name of the table and "emp_id" is the column in that table that you want to select. That's necessary, because more than one table is involved in this staement. The "department" table could also have had a column "emp_id" which could potantially have represented different information. 

If you do not want to spell out the entire table name, you can assign tables an alias (a short name) that you use to specifiy which table to take a column from. 

```
SELECT e.emp_id, e.fname, e.lname,
        d.name AS dept_name
FROM employee AS e INNER JOIN department AS d 
        ON e.dept_id = d.dept_id;
```

Note the "AS" which can be used to assign a column a more meaningful name (here "dept_name" for the column "name" in "department") as well as assigning short "handles" (here "e" and "d") for the table names. 

### The WHERE clause

So far, we have usually extracted all the data in a table or all the data in specific columns of a table. But we can also filter the table rows during the querying process. This filtering process is very similar to creating a boolean mask in Python. The WHERE clause is followed by a boolean statement and you extract only rows for which this statement is True. 

Here, we are only selecting employees whose job title is "Head Teller" from the employees table. 

In [20]:
cnx = mysql.connector.connect(**config)                # connect 
cursor = cnx.cursor()                   
    
cursor.execute("""SELECT emp_id, fname, lname, start_date, title 
                FROM employee
                WHERE title = 'Head Teller' """)       # Select four variables from "employee" only for "Head Tellers" 

results = cursor.fetchall()
    
cursor.close()                           
cnx.close()                                            # disconnect

pd.DataFrame(results, columns = ['emp_id', 'fname', 'lname', 'start_date', 'title']) # display results

,emp_id,fname,lname,start_date,title
0,6,Helen,Fleming,2004-03-17,Head Teller
1,10,Paula,Roberts,2002-07-27,Head Teller
2,13,John,Blake,2000-05-11,Head Teller
3,16,Theresa,Markham,2001-03-15,Head Teller


**Example:** Just like you can combine boolean objects in Python, you can also combine several boolean expressions using the keywords AND as well as OR in SQL. Boolean statements do not have to be equalities. For numerical data you can also use inequalities. Use parantheses to group boolean statements together. Which kind of employees is the following statement filtering for? 

```
SELECT emp_id, fname, lname, start_date, title 
    FROM employee
    WHERE (title = 'Head Teller' AND start_date > '2006-01-01')
    OR (title = 'Teller' AND start_date > '2007-01-01');
```

**Answer:** Head Tellers who started after January 1, 2006 as well as Tellers who started after Jan 1st 2007. 

**Note of Caution:** It is possible to write Python scripts that accept user input that is then used to filter the data. When you do that, you have to be extremely careful how you write the code. A "SQL Injection" is a web hacking technique in which "bad guys" try to place malicious code in SQL statements, often via web page input. 

**Example:** Suppose we want to leave it up to the user to provide the string for "title" that is used to filter the first and last names of employees in the "employee" table. Here is one (**bad!!**) way of how to write the code using string concatenation.

In [21]:
input_title = input("please provide the job title you would like to filter by: ")   # collect user input as a string
        # e.g., Teller or Head Teller or President

cnx = mysql.connector.connect(**config)                                             # connect 
cursor = cnx.cursor()                  

query = "SELECT lname, fname, title FROM employee WHERE title = '" + input_title + "'" 

print("This is your input statement:")                                              # print out the query statment with  
print(query)                                                                        # user input incorporated                                                           

cursor.execute(query)       

results = cursor.fetchall()
    
cursor.close()                          
cnx.close()                                                                         # disconnect

pd.DataFrame(results, columns = ['lname', 'fname', 'title'])                        # format table

please provide the job title you would like to filter by: Teller
This is your input statement:
SELECT lname, fname, title FROM employee WHERE title = 'Teller'


,lname,fname,title
0,Tucker,Chris,Teller
1,Parker,Sarah,Teller
2,Grossman,Jane,Teller
3,Ziegler,Thomas,Teller
4,Jameson,Samantha,Teller
5,Mason,Cindy,Teller
6,Portman,Frank,Teller
7,Fowler,Beth,Teller
8,Tulman,Rick,Teller


**Question:** what would happen, if a user entered the string 

```
anything' OR '1'='1
```

Go ahead, try it... 

In [22]:
input_title = input("please provide the job title you would like to filter by: ")   # collect user input as a string
        # e.g., Teller or Head Teller or President

cnx = mysql.connector.connect(**config)                                             # connect 
cursor = cnx.cursor()                  

query = "SELECT lname, fname, title FROM employee WHERE title = '" + input_title + "'" 

print("This is your input statement:")                                              # print out the query statment with  
print(query)                                                                        # user input incorporated                                                           

cursor.execute(query)       

results = cursor.fetchall()
    
cursor.close()                          
cnx.close()                                                                         # disconnect

pd.DataFrame(results, columns = ['lname', 'fname', 'title'])    

please provide the job title you would like to filter by: Teller
This is your input statement:
SELECT lname, fname, title FROM employee WHERE title = 'Teller'


,lname,fname,title
0,Tucker,Chris,Teller
1,Parker,Sarah,Teller
2,Grossman,Jane,Teller
3,Ziegler,Thomas,Teller
4,Jameson,Samantha,Teller
5,Mason,Cindy,Teller
6,Portman,Frank,Teller
7,Fowler,Beth,Teller
8,Tulman,Rick,Teller


And then **think about** what would happen if a user entered the following statement instead

```
Teller'; DELETE ALL FROM customers; --
```

**Note:** the "--" is SQL's way to insert a comment into a line of code. 

In [23]:
input_title = input("please provide the job title you would like to filter by: ")   # collect user input as a string
        # e.g., Teller or Head Teller or President

cnx = mysql.connector.connect(**config)                                             # connect 
cursor = cnx.cursor()                  

query = "SELECT lname, fname, title FROM employee WHERE title = '" + input_title + "'" 

print("This is your input statement:")                                              # print out the query statment with  
print(query)                                                                        # user input incorporated                                                           

# cursor.execute(query)       

# results = cursor.fetchall()
    
cursor.close()                          
cnx.close()                                                                         # disconnect

please provide the job title you would like to filter by: Teller
This is your input statement:
SELECT lname, fname, title FROM employee WHERE title = 'Teller'


In the latter case, if we were to uncomment the ```cursor.execute(query)``` statement,  ```mysql connector python``` would actually produce an error since the execute function can only execute one SQL command at a time. You could try to include a ```multi = True``` argument into your ```cursor.execute()``` function, but then you would have to load the data again from file afterwards :-)

However, SQL commands can be executed in many ways: often they are run through web-sites where input may come through the url directly or through other forms of user input. If the SQL statements are not written carefully, then they make the database vulnerable to malicious activity. 

If you're interested in reading more about these kinds of attacks, the [Wikipedia site on SQL injection](https://en.wikipedia.org/wiki/SQL_injection) is a good place to start. 

**Example:** To avoid this problem, use Python's ability to insert variables into string statements directly. 

In [24]:
input_title = input("please provide the job title you would like to filter by: ")
        # e.g., Teller or Head Teller (also try anything' OR '1'='1)

cnx = mysql.connector.connect(**config) # connect 
cursor = cnx.cursor()                   


sql = """SELECT lname, fname, title
                FROM employee
                WHERE title = %s"""
title = (input_title, )                 # provide the input as a tuple with just one element

cursor.execute(sql, title)              # a different way to incorporate user input that is less vulnerable to attack

results = cursor.fetchall()
    
cursor.close()                          
cnx.close()                             # close connection

pd.DataFrame(results, columns = ['lname', 'fname', 'title'])

please provide the job title you would like to filter by: Teller


,lname,fname,title
0,Tucker,Chris,Teller
1,Parker,Sarah,Teller
2,Grossman,Jane,Teller
3,Ziegler,Thomas,Teller
4,Jameson,Samantha,Teller
5,Mason,Cindy,Teller
6,Portman,Frank,Teller
7,Fowler,Beth,Teller
8,Tulman,Rick,Teller


### The GROUP BY clause

The GROUP BY statement allows you to group the results of a query by common values. The HAVING statement plays the same role for GROUP BY as WHERE does for SELECT. That means, it can be used as a filter for results who fulfill a specific GROUP BY condition. 

For example, in our database, the table "department" contains the names of different departments and department ID's. Whereas the employee table also contains department ID's for each employee but no department names. Suppose we want to find the names of all departments with at least two employees. 

We could do the following:
* join the department and employee tables on the common column "emp_id"
* group the results and filter by count>2
* select the department name from the result as well as the count of employees to display

In [25]:
cnx = mysql.connector.connect(**config) # connect with config-user details
cursor = cnx.cursor()                   # cursor object 
    
cursor.execute("""SELECT d.name, count(e.emp_id) AS num_employees
                    FROM department AS d INNER JOIN employee AS e
                        ON d.dept_id = e.dept_id
                    GROUP BY d.name
                    HAVING count(e.emp_id) > 2""")       

results = cursor.fetchall()
    
cursor.close()                          # close.cursor()
cnx.close()  

pd.DataFrame(results, columns = ['name', 'employee number'])

,name,employee number
0,Operations,14
1,Administration,3


**Note:** there are other ways to aggregate data from a SQL query. For instance, you may want to find the mean of all results that fulfill certain conditions. Or use MIN(), MAX(), SUM() functions. Time permitting, we'll look at those aggregating statements in more detail later. 

### The ORDER BY clause

When you filter data from a table using SELECT and WHERE statements, the results are returned not in any particular order. If you would like to return the results ordered by one or more columns, you can specify that through the ORDER clause. 

The general syntax looks like this:

```
SELECT column1, column2, ...
FROM table_name
ORDER BY column1, column2, ... ASC|DESC; 
```

where you can specify whether you want ascending (ASC) or descending (DESC) order in each column. 

**Example:** The "account" table has one row per account and contains columns "open_emp_id" (presumably the employee who opened the account) and "product_cd" (the code for the account product like checking (CHK) or saving (SAV) etc.) If you want to study employee activity in different product types, you could extract employee ID's as well as product codes and order the results first by employee and then by product code. 

In [26]:
cnx = mysql.connector.connect(**config) 
cursor = cnx.cursor()                   
    
cursor.execute("""SELECT open_emp_id, product_cd
                  FROM account
                  ORDER BY open_emp_id ASC, product_cd ASC""")       

results = cursor.fetchall()
    
cursor.close()                         
cnx.close()  

pd.DataFrame(results, columns = ['employee ID', 'product code'])

,employee ID,product code
0,1,CD
1,1,CD
2,1,CD
3,1,CD
4,1,CD
...,...,...
67,16,CHK
68,16,CHK
69,16,SAV
70,16,SAV


**Example:** You can also order by values that are derived from the data, but that are not the actual data themselves. The "RIGHT(value, n)" function returns the last n entries of the value argument. So, if you would like to order customers by the last four digits of federal ID (think SSN), then you could do that with the following code:

In [27]:
cnx = mysql.connector.connect(**config) 
cursor = cnx.cursor()                   
    
cursor.execute("""SELECT cust_id, cust_type_cd, city, state, fed_id
                  FROM customer
                  ORDER BY RIGHT(fed_id, 4)""")       

results = cursor.fetchall()
    
cursor.close()                          # close.cursor()
cnx.close()  

pd.DataFrame(results, columns = ['cust ID', 'cust_type_cd', 'city', 'state', "fed_ID"])

,cust ID,cust_type_cd,city,state,fed_ID
0,1,I,Lynnfield,MA,111-11-1111
1,10,B,Salem,NH,04-1111111
2,14,I,Lynnfield,MA,111-11-1111
3,23,B,Salem,NH,04-1111111
4,2,I,Woburn,MA,222-22-2222
5,11,B,Wilmington,MA,04-2222222
6,15,I,Woburn,MA,222-22-2222
7,24,B,Wilmington,MA,04-2222222
8,3,I,Quincy,MA,333-33-3333
9,12,B,Salem,NH,04-3333333


# Querying Multiple Tables

Frequently, you will need to combine information from two or more tables in order to gather the data you want. 
There are several different ways of merging tables in SQL. They are generally called "joins". 

* INNER JOIN
* LEFT JOIN
* RIGHT JOIN 
* OUTER JOIN

They differ by how much data you will end up with in the result table. The intersection of rows (inner), all the rows in the left table (left join), all the rows in the right table (right join) or the union of rows from both tables (outer join). Most frequently in practice you will likely need inner joins and those are the type of merger will we cover next. An inner join will merge two tables by specified common columns. Often, one of these columns is the primary key of its table whereas the other column is a foreign key in its table. The two columns do not have to have the same name, but they have to refer to the same entities. 

**Example:** You have already previously seen examples of such join statements. Recall, that in the bank example the employee table has "emp_id" as its primary key. That same table also contains employee first and last names and department IDs but not department names. The department names can be found in the "department" table which contains department IDs as well as department names. 

In [28]:
cnx = mysql.connector.connect(**config) # connect 
cursor = cnx.cursor()                   

cursor.execute(""" SELECT e.fname, e.lname, d.name
                   FROM employee AS e INNER JOIN department AS d
                       ON e.dept_id = d.dept_id""")           
                                        # SQL command 

tables = cursor.fetchall()              # fetching the result from the command line

for table in tables:                    # print results
    print(table)

cursor.close()                          
cnx.close()                             # disconnect

('Susan', 'Hawthorne', 'Operations')
('Helen', 'Fleming', 'Operations')
('Chris', 'Tucker', 'Operations')
('Sarah', 'Parker', 'Operations')
('Jane', 'Grossman', 'Operations')
('Paula', 'Roberts', 'Operations')
('Thomas', 'Ziegler', 'Operations')
('Samantha', 'Jameson', 'Operations')
('John', 'Blake', 'Operations')
('Cindy', 'Mason', 'Operations')
('Frank', 'Portman', 'Operations')
('Theresa', 'Markham', 'Operations')
('Beth', 'Fowler', 'Operations')
('Rick', 'Tulman', 'Operations')
('John', 'Gooding', 'Loans')
('Michael', 'Smith', 'Administration')
('Susan', 'Barker', 'Administration')
('Robert', 'Tyler', 'Administration')


**Note:** If there were departments whose IDs show up in one of the two tables but not in the other, then they would not be reflected in this resulting table. "INNER" is the default for joins, you could omit that word from your syntax. 

**Example:** You can also join more than two tables. For example, let's retrieve the account number and tax ID of all business accounts as well as the name (first and last) of the employee who openend the account. 

The information about whether an account is a personal or a business account is in the customer table. The customer table can be joined with the account table to find account numbers. Federal ID's are in the customer table. The name of the employee who opened the account is in the employee table, which can be joined with the account table through the account table's foreign key "open_emp_id". 

In [29]:
cnx = mysql.connector.connect(**config) # connect 
cursor = cnx.cursor()                   

cursor.execute(""" SELECT a.account_id, c.fed_id, e.fname, e.lname
                    FROM account AS a INNER JOIN customer AS c
                    ON a.cust_id = c.cust_id
                    INNER JOIN employee AS e
                    ON a.open_emp_id = e.emp_id
                    WHERE c.cust_type_cd = 'B'""")                        # SQL command  
                                        
result = cursor.fetchall()              # fetching the result from the command line

cursor.close()                          
cnx.close()    

pd.DataFrame(result, columns=['account_id', 'fed_id', 'first name', 'last name'])

,account_id,fed_id,first name,last name
0,24,04-1111111,Theresa,Markham
1,25,04-1111111,Theresa,Markham
2,85,04-1111111,Theresa,Markham
3,86,04-1111111,Theresa,Markham
4,27,04-2222222,Paula,Roberts
5,92,04-2222222,Paula,Roberts
6,28,04-3333333,Theresa,Markham
7,95,04-3333333,Theresa,Markham
8,29,04-4444444,John,Blake
9,98,04-4444444,John,Blake


**Note:** In the above example, we are combining the three tables "account", "customer", and "employee". You can choose in which order you combine the accounts. Here "accounts" is linked to both customers and employees, but employees and customers are not linked to each other. That means that you have two options: You can first join accounts and customers and then join in employees (as in the above example) or you could have first joined accounts and employees and then joined in customers. It also does not matter in your first JOIN statement, in which order you specify the tables. 

In general, the manner in which a SQL query that you write is actually executed is left to a component of your database called the "optimizer". The optimizer will try to find an efficient way to carry out the procedure specified in your statement. It is possible to control the optimizer's decisions on how to carry out a procedure by specifying "optimizer hints".

**Caution:** If your "hints" are not very good, there is a reasonable chance, that you will be making your machine work harder rather than make things more efficient. Therefore, only provide "hints" if you're really sure you know what you're doing. That will likely not be the case for most of you after 2 weeks of learning SQL. If you are interested in learning more, [read this](https://www.red-gate.com/simple-talk/sql/oracle/a-beginners-guide-to-optimizer-hints/), for example. 

## Self Joins

Sometimes, you may have to join a table to itself. Consider the employee table for instance, which contains the variables emp_ID, fname, lname and superior_emp_id. The latter is the ID of the supervisor of the employee with emp_ID. Suppose you want to create a table of all employees at Headquarters together with the names of their supervisors. 

That can be done by joining the employee table to itself. 

In [30]:
cnx = mysql.connector.connect(**config) # connect 
cursor = cnx.cursor()                   

cursor.execute(""" SELECT e.fname, e.lname, e_mgr.fname mgr_fname, e_mgr.lname mgr_lname
                     FROM employee AS e INNER JOIN employee AS e_mgr
                     ON e.superior_emp_id = e_mgr.emp_id
                     INNER JOIN branch AS b
                     ON e.assigned_branch_id=b.branch_id 
                     WHERE b.name = 'Headquarters'""")                        # SQL command  
                                        
result = cursor.fetchall()              # fetching the result from the command line

cursor.close()                          
cnx.close()    

pd.DataFrame(result, columns=['employee first name', 'employee last name', 'manager first name', 'manager last name'])

,employee first name,employee last name,manager first name,manager last name
0,Susan,Barker,Michael,Smith
1,Robert,Tyler,Michael,Smith
2,Susan,Hawthorne,Robert,Tyler
3,John,Gooding,Susan,Hawthorne
4,Helen,Fleming,Susan,Hawthorne
5,Chris,Tucker,Helen,Fleming
6,Sarah,Parker,Helen,Fleming
7,Jane,Grossman,Helen,Fleming


## Grouping and Aggregating

Data is usually stored at the level of the lowest entity of interest. In the case of the bank data, we might have data on transactions, which are nested within accounts which are nested within customers, which are nested within branches, for example. You might want to aggregate data at any of these levels: How many transactions are there on average during a certain time frame per account, or per customer, or per branch? Similarly, you could also aggregate the total amount of transactions.

If you will use SQL in your future jobs, then there is a good chance that you will be asked to aggregate data from your employers database to generate tables that show descriptive statistics for some quantity of interest. 

The GROUP BY clause (which we have encountered previously briefly when we discussed how to query data) is what you usually use to aggregate your data. 

**Example:** Suppose you want to know how many new accounts have been openend by each employee. This information is available in the account table. But it's tedious to count how often each employee appears, especially if your database is very large. 

In [31]:
cnx = mysql.connector.connect(**config) # connect 
cursor = cnx.cursor()                   

cursor.execute(""" SELECT open_emp_id
                   FROM account """)                        # SQL command  
                                        
result = cursor.fetchall()              # fetching the result from the command line

cursor.close()                          
cnx.close()    

pd.DataFrame(result, columns=['employee id'])

,employee id
0,1
1,1
2,1
3,1
4,1
...,...
67,16
68,16
69,16
70,16


**Example:** Incorporating the GROUP BY clause into your SQL syntax, will group the results by employee ID, that means the output will include each employee ID only once now. 

In [32]:
cnx = mysql.connector.connect(**config) # connect 
cursor = cnx.cursor()                   

cursor.execute(""" SELECT open_emp_id
                   FROM account 
                   GROUP BY open_emp_id""")                        # SQL command  
                                        
result = cursor.fetchall()              # fetching the result from the command line

cursor.close()                          
cnx.close()    

pd.DataFrame(result, columns=['employee id'])

,employee id
0,1
1,10
2,13
3,16


**Example:** COUNT() is an aggregate function that counts how many rows are in a group. The asterisk indicates to count all the rows in the group. 

In [33]:
cnx = mysql.connector.connect(**config) # connect 
cursor = cnx.cursor()                   

cursor.execute(""" SELECT open_emp_id, COUNT(*) 
                   FROM account 
                   GROUP BY open_emp_id""")                        # SQL command  
                                        
result = cursor.fetchall()              # fetching the result from the command line

cursor.close()                          
cnx.close()    

pd.DataFrame(result, columns=['employee id', 'number of accounts openend'])

,employee id,number of accounts openend
0,1,24
1,10,21
2,13,9
3,16,18


## Aggreate Functions in SQL

Other aggregating functions that are available in SQL include 

| Function  | Syntax  |   |
|:-:|:-:|:-:|
| Maximum   |  MAX() | Find the maximum  |
| Minimum  | MIN()  | Finds the minimum  |
| Average  | AVG()  | Finds the average value |
| Sum   | SUM()  | Finds the sum  |
|  Count | COUNT()  | Finds the number of values in a set  |

**Note:** You can use the aggregating functions with or without a GROUP BY statement. If used with a GROUP BY statement, they will return the aggregate value for each group. If used without a GROUP BY statement as in the example below, they will aggregate all the available data. 

Slide Type

Example: Find the min, max, and average balance for all checking accounts in the "accounts" table. Also count how many checking accounts there are in the table.


In [34]:
cnx = mysql.connector.connect(**config) # connect 
cursor = cnx.cursor()                   

cursor.execute(""" SELECT MIN(avail_balance) min_balance,
                    MAX(avail_balance) max_balance,
                    AVG(avail_balance) avg_balance,
                    COUNT(*) num_accounts
                    FROM account
                    WHERE product_cd = 'CHK'""")                        # SQL command  
                                        
result = cursor.fetchall()              # fetching the result from the command line

cursor.close()                          
cnx.close()    

pd.DataFrame(result, columns=['min balance', 'max balance', 'average balance', 'number'])

,min balance,max balance,average balance,number
0,122.37,38552.05,7300.800985,30


**Example:** When used in conjunction with a GROUP BY statement, the aggregate functions will aggregate data within each group defined by the GROUP BY statement separately. 

In [35]:
cnx = mysql.connector.connect(**config) # connect 
cursor = cnx.cursor()                   

cursor.execute(""" SELECT product_cd, 
                    MIN(avail_balance) min_balance,
                    MAX(avail_balance) max_balance,
                    AVG(avail_balance) avg_balance,
                    COUNT(*) num_accounts
                    FROM account
                    GROUP BY product_cd""")                        # SQL command  
                                        
result = cursor.fetchall()              # fetching the result from the command line

cursor.close()                          
cnx.close()    

pd.DataFrame(result, columns=['account type', 'min balance', 'max balance', 'average balance', 'number'])

,account type,min balance,max balance,average balance,number
0,BUS,0.00,9345.55,4672.774902,6
1,CD,1500.00,10000.00,4875.000000,12
2,CHK,122.37,38552.05,7300.800985,30
3,MM,2212.50,9345.55,5681.713216,9
4,SAV,200.00,767.77,463.940002,12
5,SBL,50000.00,50000.00,50000.000000,3
